In [2]:
import findspark

import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import histogram_numeric, col, lit
from pyspark.sql.types import IntegerType, StringType, DoubleType

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import gc

import os
import sys

findspark.init()

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName("Insurance Cross Selling").getOrCreate()
spark

In [4]:
df = spark.read.format("csv").options(
        header=True,
        inferSchema=True
    ).load("data/train.csv")
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Driving_License: integer (nullable = true)
 |-- Region_Code: double (nullable = true)
 |-- Previously_Insured: integer (nullable = true)
 |-- Vehicle_Age: string (nullable = true)
 |-- Vehicle_Damage: string (nullable = true)
 |-- Annual_Premium: double (nullable = true)
 |-- Policy_Sales_Channel: double (nullable = true)
 |-- Vintage: integer (nullable = true)
 |-- Response: integer (nullable = true)



In [18]:
df.show(10)

+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+---+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
|  0|  Male| 21|              1|       35.0|                 0|   1-2 Year|           Yes|       65101.0|               124.0|    187|       0|
|  1|  Male| 43|              1|       28.0|                 0|  > 2 Years|           Yes|       58911.0|                26.0|    288|       1|
|  2|Female| 25|              1|       14.0|                 1|   < 1 Year|            No|       38043.0|               152.0|    254|       0|
|  3|Female| 35|              1|        1.0|                 0|   1-2 Year|           Yes|        2630.0|               156.0|     76|  

In [19]:
df.summary().show()

+-------+-----------------+--------+------------------+-------------------+------------------+-------------------+-----------+--------------+------------------+--------------------+------------------+-------------------+
|summary|               id|  Gender|               Age|    Driving_License|       Region_Code| Previously_Insured|Vehicle_Age|Vehicle_Damage|    Annual_Premium|Policy_Sales_Channel|           Vintage|           Response|
+-------+-----------------+--------+------------------+-------------------+------------------+-------------------+-----------+--------------+------------------+--------------------+------------------+-------------------+
|  count|         11504798|11504798|          11504798|           11504798|          11504798|           11504798|   11504798|      11504798|          11504798|            11504798|          11504798|           11504798|
|   mean|        5752398.5|    NULL| 38.38356336199905| 0.9980219557092614| 26.41868976752134|0.46299656890977137|  

In [15]:
categorical_cols = [
    "Gender",
    "Driving_License",
    "Region_Code",
    "Previously_Insured",
    "Vehicle_Age",
    "Vehicle_Damage",
    "Policy_Sales_Channel",
    "Response",
]

In [25]:
sp_rows = 3
sp_cols = 3
sp_ids = [(r, c) for r in range(1, sp_rows + 1) for c in range(1, sp_cols + 1)]

traces = []
titles = []
sp_ind = 0

for column in df.columns:
    if column not in ["id"]:
        if column in categorical_cols:
            print(sp_ind, column)
            agg = df.groupBy(column).count().toPandas()

            if column == "Region_Code":
                traces.append(
                    (
                        go.Bar(
                            x=agg[column],
                            y=agg["count"],
                            name=column,
                            marker=dict(color="rgb(255,0,0)")
                        ),
                        sp_ids[sp_ind],
                    )
                )
            else:
                traces.append(
                    (
                        go.Bar(
                            x=agg[column],
                            y=agg["count"],
                            name=column,
                        ),
                        sp_ids[sp_ind],
                    )
                )

            titles.append(column)
            sp_ind += 1

fig = make_subplots(rows=sp_rows, cols=sp_cols, subplot_titles=titles)
for trace, sp in traces:
    fig.append_trace(trace, row=sp[0], col=sp[1])

fig.update_layout(height=1000, width=1000, title_text="Counts of categorical columns")
fig.show()
gc.collect()


0 Gender
1 Driving_License
2 Region_Code
3 Previously_Insured
4 Vehicle_Age
5 Vehicle_Damage
6 Policy_Sales_Channel
7 Response


548

In [21]:
sp_rows = 1
sp_cols = 3
sp_ids = [(r, c) for r in range(1, sp_rows + 1) for c in range(1, sp_cols + 1)]

traces = []
titles = []
sp_ind = 0

for column in df.columns:
    if column not in ["id"]:
        if column not in categorical_cols:
            print(sp_ind, column)
            bins, counts = df.select(column).rdd.flatMap(lambda x: x).histogram(10)
            positions = [((i + j) / 2) for i, j in zip(bins, bins[1:])]
            widths = [((j - i) / 2) for i, j in zip(bins, bins[1:])]
            hover_texts = [f"[{i}-{j}, {c})" for i, j, c in zip(bins, bins[1:], counts)]

            traces.append(
                (
                    go.Bar(
                        x=positions,
                        y=counts,
                        name=column,
                        hovertext=hover_texts
                    ),
                    sp_ids[sp_ind]
                )
            )
            titles.append(column)
            sp_ind += 1

fig = make_subplots(rows=sp_rows, cols=sp_cols, subplot_titles=titles)
for trace, sp in traces:
    fig.append_trace(trace, row=sp[0], col=sp[1])

fig.update_layout(
    height=500, 
    width=1000,
    bargap=0,
    bargroupgap=0,
    title_text="Histograms of numerical columns"
)
fig.show()
gc.collect()

0 Age
1 Annual_Premium
2 Vintage


459

In [29]:
no_licenses = df.filter(df.Driving_License != 1)

In [30]:
no_licenses.show(10)

+----+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
|  id|Gender|Age|Driving_License|Region_Code|Previously_Insured|Vehicle_Age|Vehicle_Damage|Annual_Premium|Policy_Sales_Channel|Vintage|Response|
+----+------+---+---------------+-----------+------------------+-----------+--------------+--------------+--------------------+-------+--------+
| 471|  Male| 51|              0|       15.0|                 1|   1-2 Year|            No|       29130.0|                26.0|    221|       0|
| 605|Female| 68|              0|        3.0|                 0|   1-2 Year|            No|       33002.0|               155.0|    107|       0|
| 640|  Male| 71|              0|       28.0|                 1|   1-2 Year|            No|       32732.0|                26.0|    257|       0|
|1463|  Male| 59|              0|       45.0|                 1|   1-2 Year|            No|       39262.0|               124.0|   

In [31]:
no_licenses.summary().show()

+-------+------------------+------+-----------------+---------------+------------------+-------------------+-----------+--------------+------------------+--------------------+------------------+--------------------+
|summary|                id|Gender|              Age|Driving_License|       Region_Code| Previously_Insured|Vehicle_Age|Vehicle_Damage|    Annual_Premium|Policy_Sales_Channel|           Vintage|            Response|
+-------+------------------+------+-----------------+---------------+------------------+-------------------+-----------+--------------+------------------+--------------------+------------------+--------------------+
|  count|             22757| 22757|            22757|          22757|             22757|              22757|      22757|         22757|             22757|               22757|             22757|               22757|
|   mean| 5755725.733400712|  NULL|64.82761348156612|            0.0|26.806652898009403|0.30917959309223536|       NULL|          NULL| 

In [32]:
sp_rows = 3
sp_cols = 3
sp_ids = [(r, c) for r in range(1, sp_rows + 1) for c in range(1, sp_cols + 1)]

traces = []
titles = []
sp_ind = 0

for column in no_licenses.columns:
    if column not in ["id"]:
        if column in categorical_cols:
            print(sp_ind, column)
            agg = no_licenses.groupBy(column).count().toPandas()

            if column == "Region_Code":
                traces.append(
                    (
                        go.Bar(
                            x=agg[column],
                            y=agg["count"],
                            name=column,
                            marker=dict(color="rgb(255,0,0)")
                        ),
                        sp_ids[sp_ind],
                    )
                )
            else:
                traces.append(
                    (
                        go.Bar(
                            x=agg[column],
                            y=agg["count"],
                            name=column,
                        ),
                        sp_ids[sp_ind],
                    )
                )

            titles.append(column)
            sp_ind += 1

fig = make_subplots(rows=sp_rows, cols=sp_cols, subplot_titles=titles)
for trace, sp in traces:
    fig.append_trace(trace, row=sp[0], col=sp[1])

fig.update_layout(height=1000, width=1000, title_text="Counts of categorical columns of No License drivers")
fig.show()
gc.collect()


0 Gender
1 Driving_License
2 Region_Code
3 Previously_Insured
4 Vehicle_Age
5 Vehicle_Damage
6 Policy_Sales_Channel
7 Response


529

In [33]:
sp_rows = 1
sp_cols = 3
sp_ids = [(r, c) for r in range(1, sp_rows + 1) for c in range(1, sp_cols + 1)]

traces = []
titles = []
sp_ind = 0

for column in no_licenses.columns:
    if column not in ["id"]:
        if column not in categorical_cols:
            print(sp_ind, column)
            bins, counts = no_licenses.select(column).rdd.flatMap(lambda x: x).histogram(10)
            positions = [((i + j) / 2) for i, j in zip(bins, bins[1:])]
            widths = [((j - i) / 2) for i, j in zip(bins, bins[1:])]
            hover_texts = [f"[{i}-{j}, {c})" for i, j, c in zip(bins, bins[1:], counts)]

            traces.append(
                (
                    go.Bar(
                        x=positions,
                        y=counts,
                        name=column,
                        hovertext=hover_texts
                    ),
                    sp_ids[sp_ind]
                )
            )
            titles.append(column)
            sp_ind += 1

fig = make_subplots(rows=sp_rows, cols=sp_cols, subplot_titles=titles)
for trace, sp in traces:
    fig.append_trace(trace, row=sp[0], col=sp[1])

fig.update_layout(
    height=500, 
    width=1000,
    bargap=0,
    bargroupgap=0,
    title_text="Histograms of numerical columns of No License Drivers"
)
fig.show()
gc.collect()

0 Age
1 Annual_Premium
2 Vintage


464

In [10]:
majority_df = df.filter(col("Response") == 0)
minority_df = df.filter(col("Response") == 1)

ratio = int(majority_df.count() / minority_df.count())

undersampled_majority = majority_df.sample(withReplacement=False, fraction=1 / ratio)
resampled_df = undersampled_majority.unionAll(minority_df)

In [13]:
resampled_df.groupby("Response").count().show()

+--------+-------+
|Response|  count|
+--------+-------+
|       0|1440495|
|       1|1415059|
+--------+-------+



In [16]:
sp_rows = 3
sp_cols = 3
sp_ids = [(r, c) for r in range(1, sp_rows + 1) for c in range(1, sp_cols + 1)]

traces = []
titles = []
sp_ind = 0

for column in resampled_df.columns:
    if column not in ["id"]:
        if column in categorical_cols:
            print(sp_ind, column)
            agg = resampled_df.groupBy(column).count().toPandas()

            if column == "Region_Code":
                traces.append(
                    (
                        go.Bar(
                            x=agg[column],
                            y=agg["count"],
                            name=column,
                            marker=dict(color="rgb(255,0,0)")
                        ),
                        sp_ids[sp_ind],
                    )
                )
            else:
                traces.append(
                    (
                        go.Bar(
                            x=agg[column],
                            y=agg["count"],
                            name=column,
                        ),
                        sp_ids[sp_ind],
                    )
                )

            titles.append(column)
            sp_ind += 1

fig = make_subplots(rows=sp_rows, cols=sp_cols, subplot_titles=titles)
for trace, sp in traces:
    fig.append_trace(trace, row=sp[0], col=sp[1])

fig.update_layout(height=1000, width=1000, title_text="Counts of categorical columns of Resampled DF")
fig.show()
gc.collect()


0 Gender
1 Driving_License
2 Region_Code
3 Previously_Insured
4 Vehicle_Age
5 Vehicle_Damage
6 Policy_Sales_Channel
7 Response


129

In [17]:
sp_rows = 1
sp_cols = 3
sp_ids = [(r, c) for r in range(1, sp_rows + 1) for c in range(1, sp_cols + 1)]

traces = []
titles = []
sp_ind = 0

for column in resampled_df.columns:
    if column not in ["id"]:
        if column not in categorical_cols:
            print(sp_ind, column)
            bins, counts = resampled_df.select(column).rdd.flatMap(lambda x: x).histogram(10)
            positions = [((i + j) / 2) for i, j in zip(bins, bins[1:])]
            widths = [((j - i) / 2) for i, j in zip(bins, bins[1:])]
            hover_texts = [f"[{i}-{j}, {c})" for i, j, c in zip(bins, bins[1:], counts)]

            traces.append(
                (
                    go.Bar(
                        x=positions,
                        y=counts,
                        name=column,
                        hovertext=hover_texts
                    ),
                    sp_ids[sp_ind]
                )
            )
            titles.append(column)
            sp_ind += 1

fig = make_subplots(rows=sp_rows, cols=sp_cols, subplot_titles=titles)
for trace, sp in traces:
    fig.append_trace(trace, row=sp[0], col=sp[1])

fig.update_layout(
    height=500, 
    width=1000,
    bargap=0,
    bargroupgap=0,
    title_text="Histograms of numerical columns of Resampled DF"
)
fig.show()
gc.collect()

0 Age
1 Annual_Premium
2 Vintage


471